## Обработчик резюме

In [532]:
import re
import os
import docx
import dateparser
import datetime

from pathlib import Path
from pymystem3 import Mystem
from pdfminer.high_level import extract_text

DATA_PATH = 'data/'

class Parser():
    def __init__(self, data_path):
        self.data_path = data_path
        self.resumes = {}
        self.final_json= {}

    # Функция для преобразования PDF-файла в текст
    def convert_pdf_to_text(self, path_to_pdf):
        with open(path_to_pdf, 'rb') as file:
            text = extract_text(file)
            return text
        
    # Функция для преобразования DOCX-файла в текст
    def convert_docx_to_text(self, path_to_docx):
        with open(path_to_docx, 'rb') as file:
            text = ""
            document = docx.Document(file)
            for paragraph in document.paragraphs:
                text += " " + paragraph.text
            return text
    
    # удаление лишних символов
    def preprocess_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^a-zA-Z0-9а-яА-Я /._,@\-?=]', '', text)
        return text.strip()
    
    # Функция для получения данных из файлов в указанной директории
    def get_data(self):
        for i, file in enumerate(os.listdir(DATA_PATH)):
            if Path(file).suffix == '.pdf':
                resume = self.convert_pdf_to_text(DATA_PATH + file)
            elif Path(file).suffix == '.docx':
                resume = self.convert_docx_to_text(DATA_PATH + file)
            else:
                resume = False
            
            if resume:
                self.resumes[f'{DATA_PATH}{file}_{i}'] = self.preprocess_text(resume)

    # Функция для получения ФИО и города
    def get_mystem_info(self, text):
        m = Mystem()

        #TODO как ускорить? объединять 10-20 пфдов в один файл и делать его анализ
        analyze = m.analyze(text)

        fio = {
            'last_name': '',
            'first_name': '',
            'middle_name': ''
        }

        city = ''
        countries = ['Россия', 'Казахстан', 'Украина', 'Беларусь'] # TODO можно заменить на список всех стран, чтобы нашелся именно город
        

        for word in analyze:
            try:
                analysis = word['analysis'][0]
            except (KeyError, IndexError):
                continue

            if 'имя' in analysis.get('gr', '') and not fio['first_name']:
                fio['first_name'] = word['text'].capitalize()
            elif 'фам' in analysis.get('gr', '') and not fio['last_name']:
                fio['last_name'] = word['text'].capitalize()
            elif 'отч' in analysis.get('gr', '') and not fio['middle_name']:
                fio['middle_name'] = word['text'].capitalize()
            elif 'гео' in analysis.get('gr', '') and city == '' and word['text'].capitalize() not in countries:
                city = word['text'].capitalize()

        fio = ' '.join(fio.values()).strip()

        return fio, city
    
    # Функция для получения даты рождения
    def get_birthday(self, text):
        birthday = re.search(r'(0?[1-9]|[12][0-9]|3[01]) (янв(?:аря)?|фев(?:раля)?|мар(?:та)?|апр(?:еля)?|мая|июн(?:я)?|июл(?:я)?|авг(?:уста)?|сен(?:тября)?|окт(?:ября)?|ноя(?:бря)?|дек(?:абря)?) ([12][0|9][0-9][0-9])', text.lower())

        if birthday:
            return dateparser.parse(birthday.group(), languages=['ru'])
        else:
            return None
    
    # Функция для получения возраста
    def get_age(self, birth_date):
        now = datetime.datetime.now()
        age = now.year - birth_date.year
        if now.month < birth_date.month or (now.month == birth_date.month and now.day < birth_date.day):
            age -= 1
        return age


    # Функция для получения почты
    def get_mail(self, text):
        mail = re.search(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}', text)

        if mail:
            return mail.group()
        else:
            return None

    # Функция для получения номера телефона (+7/7/8 - РФ номера)
    def get_phone_number(self, text):
        phone_number = re.search(r' (\+7|8|7).*?(\d{3}).*?(\d{3}).*?(\d{2}).*?(\d{2})', text)
        if phone_number:
            return phone_number.group().replace(' ', '').replace('-','')
        else:
            return None
        
    # Функция для получения гражданства
    def get_citizenship(self, text):
        words = text.upper().split(' ')
        try:
            index = words.index('ГРАЖДАНСТВО')
            val = text.split(' ')[index+1]
            return 'РФ' if 'рос' in val.lower() else val
        except ValueError:
            return ''
    
    # Функция для поиска полезных ссылок
    def get_links(self, text):
        link = re.findall(r'\b(?:http?|www\.)\S+\b', text)
        if link:
            return link
        else:
            return None


    def fill_final_json(self):

        for file, info in self.resumes.items():
            # print(file)

            fio, city = self.get_mystem_info(info)
            birthday_date = self.get_birthday(info)

            self.final_json[file] = {}
            self.final_json[file]['fio'] = fio
            self.final_json[file]['date'] = birthday_date.strftime('%d.%m.%Y')
            self.final_json[file]['age'] = self.get_age(birthday_date)
            self.final_json[file]['city'] = city
            self.final_json[file]['email'] = self.get_mail(info)
            self.final_json[file]['phone_number'] = self.get_phone_number(info)
            self.final_json[file]['citizenship'] = self.get_citizenship(info)
            self.final_json[file]['links'] = self.get_links(info)

        
            

parser = Parser(DATA_PATH)
parser.get_data()
parser.fill_final_json()
parser.final_json

{'data/Чистяков Максим Владимирович 22.47.34.pdf_1': {'fio': 'Чистяков Максим Владимирович',
  'date': '15.07.1997',
  'age': 26,
  'city': 'Москва',
  'email': 'max150797@yandex.ru',
  'phone_number': '79160525669',
  'citizenship': 'РФ',
  'links': ['https//github.com/max150797',
   'https//www.kaggle.com/max150797/competitions?tab=active']},
 'data/Нафгутдинов Рафаэль.docx_2': {'fio': 'Нафгутдинов Рафаэль',
  'date': '14.11.1995',
  'age': 28,
  'city': 'Москва',
  'email': 'max150797@yandex.ru',
  'phone_number': '79160525669',
  'citizenship': 'РФ',
  'links': None},
 'data/Резюме_Козлова Ирина.pdf_3': {'fio': 'Козлова Ирина Алексеевна',
  'date': '23.10.1996',
  'age': 27,
  'city': 'Москва',
  'email': 'ira_kozlova_00@mail.ru',
  'phone_number': '79175617480',
  'citizenship': 'РФ',
  'links': None},
 'data/Resume.pdf_4': {'fio': 'Дунаевская Мария Германовна',
  'date': '11.11.1984',
  'age': 39,
  'city': 'Москва',
  'email': 'Kupcova_marija@inbox.ru',
  'phone_number': '791611